In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=f674313b7f2b85c99677768c0c2ac2e849736937a3a84e09e711928743ba64ea
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
pip install -e git+https://github.com/jaybaird/python-bloomfilter.git#egg=pybloom

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining pybloom from git+https://github.com/jaybaird/python-bloomfilter.git#egg=pybloom
  Cloning https://github.com/jaybaird/python-bloomfilter.git to ./src/pybloom
  Running command git clone --filter=blob:none --quiet https://github.com/jaybaird/python-bloomfilter.git /content/src/pybloom
  Resolved https://github.com/jaybaird/python-bloomfilter.git to commit 2bbe01ad49965bf759e31781e6820408068862ac
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.7/272.7 kB 13.9 MB/s eta 0:00:00
  Running setup.py develop for pybloom


In [3]:
pip install warcio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 5.2 MB/s eta 0:00:00


In [4]:
pip install boto3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 100.8 MB/s eta 0:00:00


In [1]:
import os
import re
from pyspark import sql as pysql
from pyspark import SparkContext, SparkConf, SQLContext, streaming
from pyspark.sql import functions as psqlf
import pyspark
from pyspark.sql import SparkSession
import urllib
import traceback 
import pandas as pd
import pybloom
from warcio.archiveiterator import ArchiveIterator
import gzip
import boto3

In [2]:
spark = SparkSession.builder \
    .appName("myApp") \
    .config("spark.driver.memory", "64g") \
    .config("spark.executor.memory", "64g") \
    .getOrCreate()
spark

In [3]:
headers = ['WARC-Type',
    'WARC-Target-URI',
    'WARC-Date',
    'WARC-Record-ID',
    'WARC-Refers-To',
    'WARC-Block-Digest',
    'WARC-Identified-Content-Language',
    'Content-Type',
    'Content-Length'
    ]

In [4]:
ACCESS_KEY=""
SECRET_KEY=""

In [5]:
s3 = boto3.client('s3', aws_access_key_id=ACCESS_KEY, aws_secret_access_key=SECRET_KEY)
obj = s3.get_object(Bucket= 'apbd5254', Key= 'keys_novdec_22.txt')
file = obj['Body']
file = file.read()
file = file.decode("utf-8")
file = file.splitlines()

In [6]:
records = []
for i in range(20):
  obj = s3.get_object(Bucket= 'commoncrawl', Key= file[i])
  with gzip.open(obj['Body'], 'r') as stream:
      for record in ArchiveIterator(stream):
          _record = dict(record.rec_headers.headers)
          _record['Content'] = record.raw_stream.read().decode('utf-8')
          records.append(_record)
  records_df = pd.DataFrame(records)

#drop na on uri columns
records_df = records_df.dropna(axis=0, subset=['WARC-Target-URI'])
records_df['WARC-Identified-Content-Language'] = records_df['WARC-Identified-Content-Language'].fillna('not-available')

spark_df = spark.createDataFrame(records_df)
url_pattern = "https?:\\/\\/(?:www\\.)?[-a-zA-Z0-9@:%._\\+~#=]{1,256}\\.[a-zA-Z0-9()]{1,6}\\b(?:[-a-zA-Z0-9()@:%_\\+.~#?&\\/=]*)"
def get_refed_websites(content):
    urls = re.findall(url_pattern, content)
    return urls
spark_content = spark_df.select('Content')
spark_url_list = spark_content.rdd.flatMap(lambda a: get_refed_websites(a['Content']))
spark_hostnames = spark_url_list.map(lambda a: urllib.parse.urlparse(a).hostname)
spark_hostname_counts = spark_hostnames.countByValue().items()
spark_hostname_counts = sorted(spark_hostname_counts, key=(lambda a: a[1]), reverse=True)
s3.put_object(
Body=str(spark_hostname_counts), 
Bucket='apbd5254', 
Key='Popularity/'+'1' +'.txt')

{'ResponseMetadata': {'RequestId': '2YK65FT7YH6ZKKCK',
  'HostId': 'Y5tQWyt+nalkpFil6a/KtCVLf7lZ2/QK5qajkMMTSyLpeVz2X4nA90N6Vlhs6sEQZrMoVyuklrw=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'Y5tQWyt+nalkpFil6a/KtCVLf7lZ2/QK5qajkMMTSyLpeVz2X4nA90N6Vlhs6sEQZrMoVyuklrw=',
   'x-amz-request-id': '2YK65FT7YH6ZKKCK',
   'date': 'Tue, 02 May 2023 23:35:48 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"d27edd0223a208e76e863216bc15a0d2"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"d27edd0223a208e76e863216bc15a0d2"',
 'ServerSideEncryption': 'AES256'}

In [6]:
records = []
for i in range(20):
  obj = s3.get_object(Bucket= 'commoncrawl', Key= file[i+20])
  with gzip.open(obj['Body'], 'r') as stream:
      for record in ArchiveIterator(stream):
          _record = dict(record.rec_headers.headers)
          _record['Content'] = record.raw_stream.read().decode('utf-8')
          records.append(_record)
  records_df = pd.DataFrame(records)

#drop na on uri columns
records_df = records_df.dropna(axis=0, subset=['WARC-Target-URI'])
records_df['WARC-Identified-Content-Language'] = records_df['WARC-Identified-Content-Language'].fillna('not-available')

spark_df = spark.createDataFrame(records_df)
url_pattern = "https?:\\/\\/(?:www\\.)?[-a-zA-Z0-9@:%._\\+~#=]{1,256}\\.[a-zA-Z0-9()]{1,6}\\b(?:[-a-zA-Z0-9()@:%_\\+.~#?&\\/=]*)"
def get_refed_websites(content):
    urls = re.findall(url_pattern, content)
    return urls
spark_content = spark_df.select('Content')
spark_url_list = spark_content.rdd.flatMap(lambda a: get_refed_websites(a['Content']))
spark_hostnames = spark_url_list.map(lambda a: urllib.parse.urlparse(a).hostname)
spark_hostname_counts = spark_hostnames.countByValue().items()
spark_hostname_counts = sorted(spark_hostname_counts, key=(lambda a: a[1]), reverse=True)
s3.put_object(
Body=str(spark_hostname_counts), 
Bucket='apbd5254', 
Key='Popularity/'+'2' +'.txt')

{'ResponseMetadata': {'RequestId': 'FB4ZCG18W4CEJG1J',
  'HostId': 'cmfjXJVep0iPLiutwyNodqbuOrEWAzlll3U665qYiNHZp/HgM32C9YX1CQcgi7jGGVWnAUf4rfNUeAKKuvb8PA==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'cmfjXJVep0iPLiutwyNodqbuOrEWAzlll3U665qYiNHZp/HgM32C9YX1CQcgi7jGGVWnAUf4rfNUeAKKuvb8PA==',
   'x-amz-request-id': 'FB4ZCG18W4CEJG1J',
   'date': 'Tue, 02 May 2023 23:43:55 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"8dc314f629e5045af8afd3fbca6fb1be"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"8dc314f629e5045af8afd3fbca6fb1be"',
 'ServerSideEncryption': 'AES256'}

In [6]:
records = []
for i in range(20):
  obj = s3.get_object(Bucket= 'commoncrawl', Key= file[i+40])
  with gzip.open(obj['Body'], 'r') as stream:
      for record in ArchiveIterator(stream):
          _record = dict(record.rec_headers.headers)
          _record['Content'] = record.raw_stream.read().decode('utf-8')
          records.append(_record)
  records_df = pd.DataFrame(records)

#drop na on uri columns
records_df = records_df.dropna(axis=0, subset=['WARC-Target-URI'])
records_df['WARC-Identified-Content-Language'] = records_df['WARC-Identified-Content-Language'].fillna('not-available')

spark_df = spark.createDataFrame(records_df)
url_pattern = "https?:\\/\\/(?:www\\.)?[-a-zA-Z0-9@:%._\\+~#=]{1,256}\\.[a-zA-Z0-9()]{1,6}\\b(?:[-a-zA-Z0-9()@:%_\\+.~#?&\\/=]*)"
def get_refed_websites(content):
    urls = re.findall(url_pattern, content)
    return urls
spark_content = spark_df.select('Content')
spark_url_list = spark_content.rdd.flatMap(lambda a: get_refed_websites(a['Content']))
spark_hostnames = spark_url_list.map(lambda a: urllib.parse.urlparse(a).hostname)
spark_hostname_counts = spark_hostnames.countByValue().items()
spark_hostname_counts = sorted(spark_hostname_counts, key=(lambda a: a[1]), reverse=True)
s3.put_object(
Body=str(spark_hostname_counts), 
Bucket='apbd5254', 
Key='Popularity/'+'3' +'.txt')

{'ResponseMetadata': {'RequestId': 'W751MJ9A56Y89W7P',
  'HostId': 'DgX8Ilq4T/keqGDvk4E9u8oXDEFPVZ6xdHnUXQurSq0UiMoqUmUOoCENRRZGvKIbSy4o+REG7L8=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'DgX8Ilq4T/keqGDvk4E9u8oXDEFPVZ6xdHnUXQurSq0UiMoqUmUOoCENRRZGvKIbSy4o+REG7L8=',
   'x-amz-request-id': 'W751MJ9A56Y89W7P',
   'date': 'Tue, 02 May 2023 23:48:53 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"7001455aa97fc59d43639a95ab5985aa"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"7001455aa97fc59d43639a95ab5985aa"',
 'ServerSideEncryption': 'AES256'}

In [6]:
records = []
for i in range(20):
  obj = s3.get_object(Bucket= 'commoncrawl', Key= file[i+60])
  with gzip.open(obj['Body'], 'r') as stream:
      for record in ArchiveIterator(stream):
          _record = dict(record.rec_headers.headers)
          _record['Content'] = record.raw_stream.read().decode('utf-8')
          records.append(_record)
  records_df = pd.DataFrame(records)

#drop na on uri columns
records_df = records_df.dropna(axis=0, subset=['WARC-Target-URI'])
records_df['WARC-Identified-Content-Language'] = records_df['WARC-Identified-Content-Language'].fillna('not-available')

spark_df = spark.createDataFrame(records_df)
url_pattern = "https?:\\/\\/(?:www\\.)?[-a-zA-Z0-9@:%._\\+~#=]{1,256}\\.[a-zA-Z0-9()]{1,6}\\b(?:[-a-zA-Z0-9()@:%_\\+.~#?&\\/=]*)"
def get_refed_websites(content):
    urls = re.findall(url_pattern, content)
    return urls
spark_content = spark_df.select('Content')
spark_url_list = spark_content.rdd.flatMap(lambda a: get_refed_websites(a['Content']))
spark_hostnames = spark_url_list.map(lambda a: urllib.parse.urlparse(a).hostname)
spark_hostname_counts = spark_hostnames.countByValue().items()
spark_hostname_counts = sorted(spark_hostname_counts, key=(lambda a: a[1]), reverse=True)
s3.put_object(
Body=str(spark_hostname_counts), 
Bucket='apbd5254', 
Key='Popularity/'+'4' +'.txt')

{'ResponseMetadata': {'RequestId': '9R61VZ6S09TXA1EX',
  'HostId': 'Uw1YqyYFn2DrNDZo3fX8Eno8qJzz1w57gWmonLzcTz9C9YzH/xuiGEMVio4gmbRDLwR1foWUXU8=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'Uw1YqyYFn2DrNDZo3fX8Eno8qJzz1w57gWmonLzcTz9C9YzH/xuiGEMVio4gmbRDLwR1foWUXU8=',
   'x-amz-request-id': '9R61VZ6S09TXA1EX',
   'date': 'Tue, 02 May 2023 23:56:57 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"1bd4ea6430964cdec34ab6f2b377d1f5"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"1bd4ea6430964cdec34ab6f2b377d1f5"',
 'ServerSideEncryption': 'AES256'}

In [12]:
records = []
for i in range(20):
  obj = s3.get_object(Bucket= 'commoncrawl', Key= file[i+80])
  with gzip.open(obj['Body'], 'r') as stream:
      for record in ArchiveIterator(stream):
          _record = dict(record.rec_headers.headers)
          _record['Content'] = record.raw_stream.read().decode('utf-8')
          records.append(_record)
  records_df = pd.DataFrame(records)

#drop na on uri columns
records_df = records_df.dropna(axis=0, subset=['WARC-Target-URI'])
records_df['WARC-Identified-Content-Language'] = records_df['WARC-Identified-Content-Language'].fillna('not-available')

spark_df = spark.createDataFrame(records_df)
url_pattern = "https?:\\/\\/(?:www\\.)?[-a-zA-Z0-9@:%._\\+~#=]{1,256}\\.[a-zA-Z0-9()]{1,6}\\b(?:[-a-zA-Z0-9()@:%_\\+.~#?&\\/=]*)"
def get_refed_websites(content):
    urls = re.findall(url_pattern, content)
    return urls
spark_content = spark_df.select('Content')
spark_url_list = spark_content.rdd.flatMap(lambda a: get_refed_websites(a['Content']))
spark_hostnames = spark_url_list.map(lambda a: urllib.parse.urlparse(a).hostname)
spark_hostname_counts = spark_hostnames.countByValue().items()
spark_hostname_counts = sorted(spark_hostname_counts, key=(lambda a: a[1]), reverse=True)
s3.put_object(
Body=str(spark_hostname_counts), 
Bucket='apbd5254', 
Key='Popularity/'+'5' +'.txt')

{'ResponseMetadata': {'RequestId': '7HY74QEQH67P90N9',
  'HostId': 'S4ktaZN7ihm+vXMmgOOBxnwQfR1SdImsg2lVKs6bwyonClC5wC1rMECSFiorQM1ZQVzAPMDeRaU=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'S4ktaZN7ihm+vXMmgOOBxnwQfR1SdImsg2lVKs6bwyonClC5wC1rMECSFiorQM1ZQVzAPMDeRaU=',
   'x-amz-request-id': '7HY74QEQH67P90N9',
   'date': 'Wed, 03 May 2023 00:02:26 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"bb09ab752756e6b0699f4aaad9776f6e"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"bb09ab752756e6b0699f4aaad9776f6e"',
 'ServerSideEncryption': 'AES256'}

In [6]:
records = []
for i in range(20):
  obj = s3.get_object(Bucket= 'commoncrawl', Key= file[i+100])
  with gzip.open(obj['Body'], 'r') as stream:
      for record in ArchiveIterator(stream):
          _record = dict(record.rec_headers.headers)
          _record['Content'] = record.raw_stream.read().decode('utf-8')
          records.append(_record)
  records_df = pd.DataFrame(records)

#drop na on uri columns
records_df = records_df.dropna(axis=0, subset=['WARC-Target-URI'])
records_df['WARC-Identified-Content-Language'] = records_df['WARC-Identified-Content-Language'].fillna('not-available')

spark_df = spark.createDataFrame(records_df)
url_pattern = "https?:\\/\\/(?:www\\.)?[-a-zA-Z0-9@:%._\\+~#=]{1,256}\\.[a-zA-Z0-9()]{1,6}\\b(?:[-a-zA-Z0-9()@:%_\\+.~#?&\\/=]*)"
def get_refed_websites(content):
    urls = re.findall(url_pattern, content)
    return urls
spark_content = spark_df.select('Content')
spark_url_list = spark_content.rdd.flatMap(lambda a: get_refed_websites(a['Content']))
spark_hostnames = spark_url_list.map(lambda a: urllib.parse.urlparse(a).hostname)
spark_hostname_counts = spark_hostnames.countByValue().items()
spark_hostname_counts = sorted(spark_hostname_counts, key=(lambda a: a[1]), reverse=True)
s3.put_object(
Body=str(spark_hostname_counts), 
Bucket='apbd5254', 
Key='Popularity/'+'6' +'.txt')

{'ResponseMetadata': {'RequestId': 'Z76Y6S48P3MW93CZ',
  'HostId': 'PL4o08x2Y0KMAhGU+6hk1z6rFy0TxFoij7uD175a30hBeIOqrb6TK2pymM5GHJtS2MIileFNp9Th6cneIapG7Q==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'PL4o08x2Y0KMAhGU+6hk1z6rFy0TxFoij7uD175a30hBeIOqrb6TK2pymM5GHJtS2MIileFNp9Th6cneIapG7Q==',
   'x-amz-request-id': 'Z76Y6S48P3MW93CZ',
   'date': 'Wed, 03 May 2023 00:07:33 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"78efa0f45a4e19d941021e8e6f10f524"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"78efa0f45a4e19d941021e8e6f10f524"',
 'ServerSideEncryption': 'AES256'}

In [6]:
records = []
for i in range(20):
  obj = s3.get_object(Bucket= 'commoncrawl', Key= file[i+120])
  with gzip.open(obj['Body'], 'r') as stream:
      for record in ArchiveIterator(stream):
          _record = dict(record.rec_headers.headers)
          _record['Content'] = record.raw_stream.read().decode('utf-8')
          records.append(_record)
  records_df = pd.DataFrame(records)

#drop na on uri columns
records_df = records_df.dropna(axis=0, subset=['WARC-Target-URI'])
records_df['WARC-Identified-Content-Language'] = records_df['WARC-Identified-Content-Language'].fillna('not-available')

spark_df = spark.createDataFrame(records_df)
url_pattern = "https?:\\/\\/(?:www\\.)?[-a-zA-Z0-9@:%._\\+~#=]{1,256}\\.[a-zA-Z0-9()]{1,6}\\b(?:[-a-zA-Z0-9()@:%_\\+.~#?&\\/=]*)"
def get_refed_websites(content):
    urls = re.findall(url_pattern, content)
    return urls
spark_content = spark_df.select('Content')
spark_url_list = spark_content.rdd.flatMap(lambda a: get_refed_websites(a['Content']))
spark_hostnames = spark_url_list.map(lambda a: urllib.parse.urlparse(a).hostname)
spark_hostname_counts = spark_hostnames.countByValue().items()
spark_hostname_counts = sorted(spark_hostname_counts, key=(lambda a: a[1]), reverse=True)
s3.put_object(
Body=str(spark_hostname_counts), 
Bucket='apbd5254', 
Key='Popularity/'+'7' +'.txt')

{'ResponseMetadata': {'RequestId': 'TWV6W63MZP1TAJTN',
  'HostId': '42Gdg3TM3v6OXSdFZQGF7RqLPZlqzFumMjQMVQfth18e/judoIARbF2DbvCRs4WAJE7bMRGIt02ltRrdVoxeSnTCcXGY3CxbSYXw2k7hnx8=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '42Gdg3TM3v6OXSdFZQGF7RqLPZlqzFumMjQMVQfth18e/judoIARbF2DbvCRs4WAJE7bMRGIt02ltRrdVoxeSnTCcXGY3CxbSYXw2k7hnx8=',
   'x-amz-request-id': 'TWV6W63MZP1TAJTN',
   'date': 'Wed, 03 May 2023 00:12:47 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"7eb7a44dca870a44ce3a4c81e3402bb7"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"7eb7a44dca870a44ce3a4c81e3402bb7"',
 'ServerSideEncryption': 'AES256'}

In [6]:
records = []
for i in range(20):
  obj = s3.get_object(Bucket= 'commoncrawl', Key= file[i+140])
  with gzip.open(obj['Body'], 'r') as stream:
      for record in ArchiveIterator(stream):
          _record = dict(record.rec_headers.headers)
          _record['Content'] = record.raw_stream.read().decode('utf-8')
          records.append(_record)
  records_df = pd.DataFrame(records)

#drop na on uri columns
records_df = records_df.dropna(axis=0, subset=['WARC-Target-URI'])
records_df['WARC-Identified-Content-Language'] = records_df['WARC-Identified-Content-Language'].fillna('not-available')

spark_df = spark.createDataFrame(records_df)
url_pattern = "https?:\\/\\/(?:www\\.)?[-a-zA-Z0-9@:%._\\+~#=]{1,256}\\.[a-zA-Z0-9()]{1,6}\\b(?:[-a-zA-Z0-9()@:%_\\+.~#?&\\/=]*)"
def get_refed_websites(content):
    urls = re.findall(url_pattern, content)
    return urls
spark_content = spark_df.select('Content')
spark_url_list = spark_content.rdd.flatMap(lambda a: get_refed_websites(a['Content']))
spark_hostnames = spark_url_list.map(lambda a: urllib.parse.urlparse(a).hostname)
spark_hostname_counts = spark_hostnames.countByValue().items()
spark_hostname_counts = sorted(spark_hostname_counts, key=(lambda a: a[1]), reverse=True)
s3.put_object(
Body=str(spark_hostname_counts), 
Bucket='apbd5254', 
Key='Popularity/'+'8' +'.txt')

{'ResponseMetadata': {'RequestId': '6Y8WK1SF0YSARVZ5',
  'HostId': '3C6vI9apdl6MMUzyPMrUy8qgV9MpYzGNO6Kh2BXEgKfwM4taPhFhpXurI3t3WfYyWaXaqqOnjKM=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '3C6vI9apdl6MMUzyPMrUy8qgV9MpYzGNO6Kh2BXEgKfwM4taPhFhpXurI3t3WfYyWaXaqqOnjKM=',
   'x-amz-request-id': '6Y8WK1SF0YSARVZ5',
   'date': 'Wed, 03 May 2023 00:18:12 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"76cc0ceb41687bc4650f4d4f21f92cdd"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"76cc0ceb41687bc4650f4d4f21f92cdd"',
 'ServerSideEncryption': 'AES256'}

In [6]:
records = []
for i in range(20):
  obj = s3.get_object(Bucket= 'commoncrawl', Key= file[i+160])
  with gzip.open(obj['Body'], 'r') as stream:
      for record in ArchiveIterator(stream):
          _record = dict(record.rec_headers.headers)
          _record['Content'] = record.raw_stream.read().decode('utf-8')
          records.append(_record)
  records_df = pd.DataFrame(records)

#drop na on uri columns
records_df = records_df.dropna(axis=0, subset=['WARC-Target-URI'])
records_df['WARC-Identified-Content-Language'] = records_df['WARC-Identified-Content-Language'].fillna('not-available')

spark_df = spark.createDataFrame(records_df)
url_pattern = "https?:\\/\\/(?:www\\.)?[-a-zA-Z0-9@:%._\\+~#=]{1,256}\\.[a-zA-Z0-9()]{1,6}\\b(?:[-a-zA-Z0-9()@:%_\\+.~#?&\\/=]*)"
def get_refed_websites(content):
    urls = re.findall(url_pattern, content)
    return urls
spark_content = spark_df.select('Content')
spark_url_list = spark_content.rdd.flatMap(lambda a: get_refed_websites(a['Content']))
spark_hostnames = spark_url_list.map(lambda a: urllib.parse.urlparse(a).hostname)
spark_hostname_counts = spark_hostnames.countByValue().items()
spark_hostname_counts = sorted(spark_hostname_counts, key=(lambda a: a[1]), reverse=True)
s3.put_object(
Body=str(spark_hostname_counts), 
Bucket='apbd5254', 
Key='Popularity/'+'9' +'.txt')

{'ResponseMetadata': {'RequestId': 'T7J5P8X4BRJQ8E1Y',
  'HostId': 'm2h5IrQUSVN3wCWLdBxEp3krPIhhDIjSg+daHdhWI2CDdj+f1R0oPGhoiMBwdWBNJVq8w3ciKLE=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'm2h5IrQUSVN3wCWLdBxEp3krPIhhDIjSg+daHdhWI2CDdj+f1R0oPGhoiMBwdWBNJVq8w3ciKLE=',
   'x-amz-request-id': 'T7J5P8X4BRJQ8E1Y',
   'date': 'Wed, 03 May 2023 00:23:43 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"74d25b4affede7570abff7f46a67909f"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"74d25b4affede7570abff7f46a67909f"',
 'ServerSideEncryption': 'AES256'}

In [6]:
records = []
for i in range(20):
  obj = s3.get_object(Bucket= 'commoncrawl', Key= file[i+180])
  with gzip.open(obj['Body'], 'r') as stream:
      for record in ArchiveIterator(stream):
          _record = dict(record.rec_headers.headers)
          _record['Content'] = record.raw_stream.read().decode('utf-8')
          records.append(_record)
  records_df = pd.DataFrame(records)

#drop na on uri columns
records_df = records_df.dropna(axis=0, subset=['WARC-Target-URI'])
records_df['WARC-Identified-Content-Language'] = records_df['WARC-Identified-Content-Language'].fillna('not-available')

spark_df = spark.createDataFrame(records_df)
url_pattern = "https?:\\/\\/(?:www\\.)?[-a-zA-Z0-9@:%._\\+~#=]{1,256}\\.[a-zA-Z0-9()]{1,6}\\b(?:[-a-zA-Z0-9()@:%_\\+.~#?&\\/=]*)"
def get_refed_websites(content):
    urls = re.findall(url_pattern, content)
    return urls
spark_content = spark_df.select('Content')
spark_url_list = spark_content.rdd.flatMap(lambda a: get_refed_websites(a['Content']))
spark_hostnames = spark_url_list.map(lambda a: urllib.parse.urlparse(a).hostname)
spark_hostname_counts = spark_hostnames.countByValue().items()
spark_hostname_counts = sorted(spark_hostname_counts, key=(lambda a: a[1]), reverse=True)
s3.put_object(
Body=str(spark_hostname_counts), 
Bucket='apbd5254', 
Key='Popularity/'+'10' +'.txt')

{'ResponseMetadata': {'RequestId': '3FPN1W0C5HFGW682',
  'HostId': 'fBrEuefWBC+BsRul+HNIaz87xe+i+Vd/YGfay4Qa4Lf636L0/8Jc6Du6EyUms53q8uHXKOilT8A=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'fBrEuefWBC+BsRul+HNIaz87xe+i+Vd/YGfay4Qa4Lf636L0/8Jc6Du6EyUms53q8uHXKOilT8A=',
   'x-amz-request-id': '3FPN1W0C5HFGW682',
   'date': 'Wed, 03 May 2023 00:29:03 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"afac726b7a7a0598056d0fa4b3f7fb24"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"afac726b7a7a0598056d0fa4b3f7fb24"',
 'ServerSideEncryption': 'AES256'}